In [1]:
# Dependencies.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

import gensim
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re # For remove_url function.

E:\Anaconda3-5.2.0\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [21]:
# TEMP
from nltk import word_tokenize
from nltk import pos_tag


text = tweets['text'].values

sentences = []
for sent in text:
    sent = remove_url(sent)
    sent = word_tokenize(sent)
    sent = pos_tag(sent)
    sentences.append(sent)

In [22]:
sentences[:10]

[[('Population', 'NN'), ('Game', 'NNP'), ('-', ':'), ('Lithuania', 'NN')],
 [('I', 'PRP'),
  ('tried', 'VBD'),
  ('even', 'RB'),
  ('inviting', 'VBG'),
  ('from', 'IN'),
  ('friends', 'NNS'),
  ('in', 'IN'),
  ('spain', 'NN'),
  (',', ','),
  ('lithuania', 'NN'),
  ('and', 'CC'),
  ('the', 'DT'),
  ('phillipines', 'NNS'),
  ('without', 'IN'),
  ('any', 'DT'),
  ('success', 'NN'),
  (':', ':'),
  ('-', ':'),
  ('(', '(')],
 [('Extreme', 'JJ'),
  ('Crafts', 'NNP'),
  ('Exhibit', 'NNP'),
  ('in', 'IN'),
  ('Lithuania', 'NNP')],
 [('Good', 'JJ'),
  ('night', 'NN'),
  ('!', '.'),
  ('tomorow', 'NN'),
  ('meeting', 'NN'),
  ('with', 'IN'),
  ('ltu', 'NN'),
  ('technologies', 'NNS'),
  ('and', 'CC'),
  ('then', 'RB'),
  ('bahut', 'VB'),
  ('team', 'NN'),
  ('.', '.')],
 [('Roaming', 'VBG'),
  ('terminal', 'JJ'),
  ('7', 'CD'),
  ('@', 'JJ'),
  ('JFK', 'NNP'),
  ('.', '.'),
  ('Board', 'NNP'),
  ('in', 'IN'),
  ('2', 'CD'),
  ('hrs', 'NN'),
  ('.', '.'),
  ('Lithuania', 'NNP'),
  ('or', 'CC'),

In [2]:
# Loading the data and removing useless columns.
tweets = pd.read_json('twitter_data.json')
tweets = tweets.drop(['html', 'url'], axis=1)

In [3]:
def remove_url(text):
    '''
    Removes any url from a string.
    
    Returns the same string format.
    
    -----
    Parameters
    
    text (str): self explanatory.
    '''
    sent = text.split()
    sent = [re.sub(r'^https?:\/\/.*[\r\n]*', '', word, flags=re.MULTILINE) for word in sent]  
    return(' '.join(sent))

In [4]:
def less_than_three(text):
    '''
    Removes words with less <3 letters.
    
    Returns the same string format.
    
    ------
    Parameters
    
    text (str): self-explanatory
    '''
    sent = text.split()
    sent = [word for word in sent if len(word) >= 3]
    return(' '.join(sent))

In [43]:
# Preprocess raw tweets.
text = tweets['text'].values
lemmatizer = WordNetLemmatizer()
sw = stopwords.words('english')

sentences = []
for sent in text:
    sent = remove_url(sent)
    sent = less_than_three(sent)
    sent = gensim.utils.simple_preprocess(sent)
    sent = [word for word in sent if word not in sw]
    sent = [lemmatizer.lemmatize(word) for word in sent]
    sentences.append(sent)
    

In [18]:
# Training word2vec.
model = gensim.models.Word2Vec(sentences) # No params for now.

In [27]:
model.most_similar('lithuania', topn=30)

E:\Anaconda3-5.2.0\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
E:\Anaconda3-5.2.0\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('krakow', 0.4063587784767151),
 ('finland', 0.39320796728134155),
 ('polan', 0.3907938003540039),
 ('estonian', 0.3847597539424896),
 ('lith', 0.38221612572669983),
 ('latvian', 0.3808610141277313),
 ('tallinn', 0.37880319356918335),
 ('dubrovnik', 0.3717016577720642),
 ('cont', 0.36985713243484497),
 ('whereas', 0.3658159673213959),
 ('bialystok', 0.36387205123901367),
 ('campaign', 0.3638075590133667),
 ('warsaw', 0.363118439912796),
 ('stockholm', 0.3612903654575348),
 ('lithuani', 0.36052843928337097),
 ('previously', 0.3604913353919983),
 ('unknown', 0.3603193759918213),
 ('linz', 0.35987526178359985),
 ('belorussia', 0.3580518364906311),
 ('near', 0.3565604090690613),
 ('ended', 0.3545544743537903),
 ('talinn', 0.3507489562034607),
 ('riga', 0.35004663467407227),
 ('likely', 0.349974662065506),
 ('oslo', 0.3492653965950012),
 ('rome', 0.3490906357765198),
 ('dublin', 0.34837979078292847),
 ('western', 0.3478650450706482),
 ('visited', 0.343481183052063),
 ('olive', 0.3419618010

In [ ]:
# Creating a dictionary.
dictionary = gensim.corpora.Dictionary(sentences)

# Filtering the extremes.
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [49]:
# Creating a Bag of Words model.
bow_corpus = [dictionary.doc2bow(sent) for sent in sentences]

# Creating a TF-IDF model.
# "tf–idf or TFIDF, short for term frequency–inverse document frequency, is a numerical statistic 
# that is intended to reflect how important a word is to a document in a collection or corpus."
tfidf = gensim.models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [57]:
# Creating LDA using BoW.
lda_model_bow = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

for idx, topic in lda_model_bow.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

NameError: name 'lda_model' is not defined

In [59]:
# Creating LDA using TF-IDF
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.010*"vilnius" + 0.010*"twitter" + 0.010*"com" + 0.008*"pic" + 0.006*"eurovision" + 0.005*"good" + 0.005*"travel" + 0.005*"ltu" + 0.005*"old" + 0.005*"liam"
Topic: 1 Word: 0.009*"lithuaniabot" + 0.008*"hot" + 0.008*"kiss" + 0.007*"sex" + 0.007*"bot" + 0.007*"twitter" + 0.007*"com" + 0.006*"let" + 0.006*"eurovision" + 0.006*"pic"
Topic: 2 Word: 0.006*"baltic" + 0.006*"sharing" + 0.006*"com" + 0.005*"suaju" + 0.005*"twitter" + 0.005*"win" + 0.005*"grip" + 0.005*"example" + 0.005*"trump" + 0.005*"negotiable"
Topic: 3 Word: 0.021*"eurovision" + 0.021*"blind" + 0.011*"love" + 0.010*"sweden" + 0.008*"spain" + 0.007*"germany" + 0.007*"like" + 0.007*"norway" + 0.006*"poland" + 0.006*"estonia"
Topic: 4 Word: 0.010*"nato" + 0.007*"baltic" + 0.007*"russia" + 0.006*"military" + 0.006*"biden" + 0.006*"com" + 0.006*"latvia" + 0.006*"twitter" + 0.005*"poland" + 0.005*"ukraine"
Topic: 5 Word: 0.028*"love" + 0.028*"eurovision" + 0.026*"come" + 0.022*"please" + 0.021*"vote" + 0.012*"foll

In [61]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda_model_bow, corpus_tfidf, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

E:\Anaconda3-5.2.0\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))
